## EY Datawave Challenge Code

**Simple rule**: 
- all "df_xx" types are pd.DataFrame
- "xx_data" are usually NumPy arrays

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import normalize
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

pd.set_option('display.max_columns', None)

# Cool point got from here

https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

In [2]:
# # fix random seed for reproducibility
# seed = 7
# np.random.seed(seed)

# Create a Callback

In [4]:
class MyCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('acc') >= 0.9:
            print("Reached 90% acc so cancelling training!")
            self.model.stop_training = True

reach_90acc = MyCallback()

# Metric for F1

https://medium.com/@thongonary/how-to-compute-f1-score-for-each-epoch-in-keras-a1acd17715a2

In [6]:
class F1(keras.callbacks.Callback):
    def __init__(self, val_data):
        super().__init__()
        self.validation_data = val_data
        
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(self.validation_data[0]))).round()
        val_targ = self.validation_data[1]
        _val_f1 = f1_score(val_targ, val_predict)
        _val_recall = recall_score(val_targ, val_predict)
        _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print ("— val_f1: %f — val_precision: %f — val_recall %f" %(_val_f1, _val_precision, _val_recall))
        return

# Read the training Data

df is training data + label

In [7]:
#read training data
df_train = pd.read_csv("/Users/Godwithus/Desktop/EY/data_train.csv", low_memory=False) #nrows = integer
df_train = df_train.loc[:,'hash':'y_exit']
df_train.fillna('', inplace=True)

# Read the test Data

df_test is test data + label

In [8]:
#read test data
df_test = pd.read_csv("/Users/Godwithus/Desktop/EY/data_test.csv", low_memory=False)
df_test = df_test.loc[:,'hash':'y_exit']
df_test.fillna('', inplace=True)

# Pred and Eval

First, change the time variables into pd.timedelta.

Then, divide the df_test into 2 subparts:

1. df_test_eval: Used for evaluating test accuracy (where x_exit value exists)
2. df_test_pred: the data we want to predict from (where x_exit value do not exist)

In [9]:
#divide the test data into pred and eval
df_test['time_entry']=pd.to_timedelta(df_test['time_entry'])
df_test['time_exit']=pd.to_timedelta(df_test['time_exit'])

## Some look at the data

In [10]:
#debugging
# print(df_test_pred)
# print(df_test_eval)

print (df_train.info())
print (df_test.info())

df_train.describe()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814262 entries, 0 to 814261
Data columns (total 11 columns):
hash             814262 non-null object
trajectory_id    814262 non-null object
time_entry       814262 non-null object
time_exit        814262 non-null object
vmax             814262 non-null object
vmin             814262 non-null object
vmean            814262 non-null object
x_entry          814262 non-null float64
y_entry          814262 non-null float64
x_exit           814262 non-null float64
y_exit           814262 non-null float64
dtypes: float64(4), object(7)
memory usage: 68.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202937 entries, 0 to 202936
Data columns (total 11 columns):
hash             202937 non-null object
trajectory_id    202937 non-null object
time_entry       202937 non-null timedelta64[ns]
time_exit        202937 non-null timedelta64[ns]
vmax             202937 non-null object
vmin             202937 non-null object
vmean            2

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,07:04:31,07:08:32,,,,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,07:20:34,07:25:42,,,,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,07:53:32,08:03:25,,,,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,08:17:50,08:37:23,,,,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,14:38:09,14:38:09,,,,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07


# Prepare the Training Data

choose the features: 

*for rn, I cannot handle velocity since there are NaN.*

Change the time values into float, by dividing into minutes.

Finally, store train_data as NumPy arrays, and normalize them.

In [11]:
# 1. Prepare time stayed in the city hall (seconds)
df_train['time_entry_seconds'] = pd.to_timedelta(df_train['time_entry']).dt.total_seconds()
df_train['time_exit_seconds']=pd.to_timedelta(df_train['time_exit']).dt.total_seconds()
df_train['time_stayed_seconds']=df_train['time_exit_seconds']-df_train['time_entry_seconds']

df_train.head()

# 2. prepare whether entry point is in cityhall
x_in_city = (df_train['x_entry'] >=3750901.5068) & (df_train['x_entry']<=3770901.5068)
y_in_city = (df_train['y_entry'] >= -19268905.6133) & (df_train['y_entry'] <= -19208905.6133)

df_train['entry_inside'] = 1*(x_in_city & y_in_city)

train_data=df_train.loc[:,['time_stayed_seconds','entry_inside']].values

df_train.entry_inside.describe()
df_train.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,time_entry_seconds,time_exit_seconds,time_stayed_seconds,entry_inside
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,07:04:31,07:08:32,,,,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,25471.0,25712.0,241.0,0
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,07:20:34,07:25:42,,,,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,26434.0,26742.0,308.0,0
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,07:53:32,08:03:25,,,,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,28412.0,29005.0,593.0,0
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,08:17:50,08:37:23,,,,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,29870.0,31043.0,1173.0,0
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,14:38:09,14:38:09,,,,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,52689.0,52689.0,0.0,0


# prepare Train Labels

prepare the label for training:

x_exit and y_exit values have to be within certain range. Do each of the comparison and store the value as 0 or 1 in train_label NumPy array.

In [12]:
#prepare training label

target_x = (df_train['x_exit']>=3750901.5068) & (df_train['x_exit']<=3770901.5068)
target_y = (df_train['y_exit']>=-19268905.6133) & (df_train['y_exit']<=-19208905.6133)

train_label = 1*(target_x & target_y)

df_train['train_label'] = train_label


train_label = train_label.values

# train_label = to_categorical(train_label)

# Prepare the Test Data

choose the features: 

*velocty for pred data is weired

Change the time values into float, by dividing into minutes.

Finally, store test_data_pred and test_data_eval as NumPy arrays, and normalize them.

In [13]:
#pred

df_test['time_entry_seconds'] = pd.to_timedelta(df_test['time_entry']).dt.total_seconds()
df_test['time_exit_seconds']=pd.to_timedelta(df_test['time_exit']).dt.total_seconds()
df_test['time_stayed_seconds']=df_test['time_exit_seconds']-df_test['time_entry_seconds']

x_in_city = (df_test['x_entry'] >=3750901.5068) & (df_test['x_entry']<=3770901.5068)
y_in_city = (df_test['y_entry'] >= -19268905.6133) & (df_test['y_entry'] <= -19208905.6133)

df_test['entry_inside'] = 1*(x_in_city & y_in_city)


# Divide Pred and Eval

In [14]:
df_test_pred = df_test[df_test['x_exit'] == ""]
df_test_eval = df_test[df_test['x_exit'] != ""]

# Prepare the features

In [15]:
test_data_pred = df_test_pred.loc[:,['time_stayed_seconds','entry_inside']]
test_data_pred = test_data_pred.values

test_data_eval = df_test_eval.loc[:,['time_stayed_seconds','entry_inside']]
test_data_eval = test_data_eval.values

# prepare Test Labels

prepare the label for test:

x_exit and y_exit values from df_test_eval (not pred). 0 or 1 in train_label NumPy array.

In [16]:
#prepare test label

target_x = (df_test_eval['x_exit']>=3750901.5068) & (df_test_eval['x_exit']<=3770901.5068)
target_y = (df_test_eval['y_exit']>=-19268905.6133) & (df_test_eval['y_exit']<=-19208905.6133)

test_label = 1*(target_x & target_y)

df_test_eval['test_label'] = test_label

test_label = test_label.values

df_test_eval.head()

/Users/Godwithus/anaconda3/envs/tf_env/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,time_entry_seconds,time_exit_seconds,time_stayed_seconds,entry_inside,test_label
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,11:43:17,11:50:17,,,,3.773413e+06,-1.909828e+07,3.77311e+06,-1.91451e+07,42197.0,42617.0,420.0,0,0
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,12:21:37,12:21:37,0,0,0,3.773199e+06,-1.914354e+07,3.7732e+06,-1.91435e+07,44497.0,44497.0,0.0,0,0
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,12:34:27,13:14:11,,,,3.763760e+06,-1.921342e+07,3.77176e+06,-1.91109e+07,45267.0,47651.0,2384.0,1,0
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,13:25:33,13:43:13,,,,3.773385e+06,-1.911344e+07,3.77313e+06,-1.91447e+07,48333.0,49393.0,1060.0,0,0
5,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_0,08:08:23,08:20:08,17.29,17.29,17.29,3.771380e+06,-1.933274e+07,3.76993e+06,-1.93409e+07,29303.0,30008.0,705.0,0,0


# Custom F1 loss function

In [17]:
def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

# define Keras NN model

binary softmax, but categorical_crossentropy loss. *can improve loss, optimizer, layer*

In [18]:
#train_data = train_data.reshape(814262, 7,1)
#define model
model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

#complile the model
model.compile(optimizer='Adam',
              loss='binary_crossentropy',
              metrics=['accuracy']) 

#fit the model
f1 = F1((test_data_eval, test_label))
history = model.fit(train_data, train_label, epochs=4, \
                    validation_data=(test_data_eval, test_label), callbacks=[reach_90acc, f1]) #, f1

Instructions for updating:
Colocations handled automatically by placer.
Train on 814262 samples, validate on 169422 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/4
814262/814262 [==============================] - 89s 109us/sample - loss: 2.8682 - acc: 0.8198 - val_loss: 1.0578 - val_acc: 0.9330
Epoch 2/4
336096/814262 [===========>..................] - ETA: 47s - loss: 1.0677 - acc: 0.9334

KeyboardInterrupt: 

# Evaluation of the Model

print the summary and test accuracy

In [ ]:
#test_data_eval = test_data_eval.reshape(169422, 7, 1)

#evaluate the accuracy of the model
model.summary()

test_loss, test_acc = model.evaluate(test_data_eval, test_label)

print('Test accuracy:', test_acc)

# Plot the model's Learning Curve

https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

In [ ]:
print(history.history.keys())

# summarize history for acc
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Predict the Data

Predict the test_data_pred and if the p(xi) is over 0.5, save it as 1, otherwise 0. Predictions is the NumPy array saving the result. Formulate pd.DataFrame from df_testPred['trajectory_id'] and predictions ('target') so that the output DataFrame is in ['id', 'target'] format.

In [ ]:
#test_data_pred = test_data_pred.reshape(33515, 7, 1)

#predict and handle output
predictions = model.predict(test_data_pred)

print(predictions)
predictions = (predictions >= 0.5) *1

id = pd.DataFrame(df_test_pred['trajectory_id'])

target = pd.DataFrame(predictions)
# target.columns = ['zeros','target']
# target = target['target']
target.columns = ['target']

output = pd.concat([id.reset_index(drop=True),target.reset_index(drop=True)], axis=1)
output.columns = ['id', 'target']
output.to_csv("/Users/Godwithus/Desktop/EY/try_1.csv", index=False)

In [ ]:
#debugging
print("test_data_pred", test_data_pred.shape)
print("predictions", predictions.shape)
print("df_test_pred", df_test_pred.shape)
print("id", id.shape)
print("target", target.shape)
print("output", output.shape)

print(output)

print(target.sum()/np.size(target, 0))